# Import Iib use for project

In [1]:
import pandas as pd
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import *
from sqlalchemy.sql import text

import os
import sys
# import the connect library for psycopg2
import psycopg2
# import the error handling libraries for psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras


# Explore Customer data from csv

In [2]:
df_custom = pd.read_csv('./DE-Test/Customer.csv')

In [3]:
df_custom

,customer_id,first_name,last_name,address,city,state,zip,email,phone
0,10005,Martie,Feasby,8915 Mitchell Place,Grand Junction,CO,81505,mfeasbyaf@printfriendly.com,970-222-2301
1,10006,Tiertza,Swainson,689 Drewry Avenue,Saint Paul,MN,55166,tswainsonqz@gmail.com,651-259-9540
2,10012,Nana,Di Lucia,3 Village Green Circle,Boca Raton,FL,33499,ndiluciadc@statcounter.com,561-923-6152
3,10014,Pierrette,Hassent,081 Mendota Pass,Madison,WI,53710,phassent3u@netscape.com,608-241-2936
4,10015,Britta,MacSkeaghan,8 Di Loreto Park,Buffalo,NY,14210,bmacskeaghanae@delicious.com,716-947-1700
...,...,...,...,...,...,...,...,...,...
7037,59995,Vivianna,Milham,564 Sycamore Parkway,Mesa,AZ,85205,vmilhami6@opensource.org,602-237-8457
7038,59996,Jacinda,Wrightson,6774 Moose Alley,Memphis,TN,38104,jwrightson7a@gmpg.org,901-513-6554
7039,59997,D'arcy,Lokier,384 Esker Way,Norfolk,VA,23551,dlokierod@gmail.com,757-365-9107
7040,59999,Clemmy,Danniell,47931 Saint Paul Way,Dallas,TX,75205,cdanniellev@opensource.org,972-370-1085


# Explore Netsale data from csv

In [4]:
df_netSale = pd.read_csv('./DE-Test/Netsale.csv')

In [5]:
#find null value
df_netSale.isnull().sum()

netsale_id     0
sale_time      0
customer_id    0
shipping       0
tax            9
total          9
dtype: int64

In [6]:
#cleaning data null value
df_netSale['tax'] = df_netSale['tax'].fillna(0)
df_netSale['total'] = df_netSale['total'].fillna(0)

In [7]:
df_netSale

,netsale_id,sale_time,customer_id,shipping,tax,total
0,10531095,1/1/21 1:03 AM,13467,16.36,4.91,103.07
1,10531270,1/1/21 9:19 AM,19177,0.00,10.57,194.37
2,10531551,1/1/21 10:55 AM,55122,13.99,5.07,89.01
3,10531593,1/1/21 11:10 AM,22113,0.00,14.11,239.81
4,10531612,1/1/21 11:15 AM,22793,5.00,0.72,20.17
...,...,...,...,...,...,...
8322,11364414,12/30/21 4:19 PM,10166,0.00,10.58,173.28
8323,11364601,12/30/21 5:02 PM,49586,0.00,17.03,300.88
8324,11364659,12/30/21 5:22 PM,50746,0.00,8.02,141.72
8325,11364889,12/30/21 7:27 PM,14378,0.00,9.17,172.92


# Explore Product data from csv

In [8]:
df_product = pd.read_csv('./DE-Test/Product.csv')

In [9]:
df_product

,product_id,name,in_store,embellishment_type,fabric,fit,graphic_type,origin,season_code,vendor,color_code,single_sku_catalog_entry,catalog,category,subcategory,current_price
0,13029606,White ^un^ Coaches Flex Slouch Hat,True,Embroidered/Sewn,97% Polyester/3% Elastane,NaN,SIDELINE,Imported,Fall,Adidas,White,103000000.0,Clothing & Accessories,Men,Hats,28.00
1,13194142,^color^ ^un^ Amplifier Short Sleeve T-Shirt,True,Screen Print,100% Cotton,NaN,SIDELINE,Imported,Fall,Adidas,^color^,NaN,Clothing & Accessories,Men,T-Shirts & Tanks,25.00
2,13547940,^un^ Youth Girls' Burnout Short Sleeve T-Shirt,True,Screen Print,51% Cotton/49% Polyester,Youth,Trend/Fashion Graphic,Imported,Basic Non-Seasonal,Wes & Willy LLC,NaN,NaN,Clothing & Accessories,Kids,Youth,26.00
3,15262883,Fan Brander Black Apple Air Pod Leatherette ca...,False,NaN,NaN,NaN,NaN,NaN,DSV Direct Ship from Vendor,Branded Logistics LLC,NaN,100000000.0,Electronics,Cell Phones,Phone Cases,29.99
4,15281446,Black/White ^un^ Women's Snap Jacket,True,Screen Print,100% Polyester,Activewear,SIDELINE,Imported,Fall,Adidas,Black/White,NaN,Clothing & Accessories,Women,Outerwear,70.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,99659630,^un^ Youth Long Sleeve T-Shirt,True,Screen Print,100% Cotton,Youth,Official Seal,Imported,Basic Non-Seasonal,Champion Products,NaN,NaN,Clothing & Accessories,Kids,Youth,22.00
1596,99721021,^un^ School of Business Adjustable Hat,True,Embroidered/Sewn,100% Cotton,NaN,School of,Imported,Basic Non-Seasonal,Legacy Apparel,^color^,99721137.0,Clothing & Accessories,Men,Hats,22.00
1597,99721022,^un^ Twill Hat,True,Embroidered/Sewn,100% Cotton,NaN,Wordmark,Imported,Basic Non-Seasonal,Legacy Apparel,Lemon,99721056.0,Clothing & Accessories,Men,Hats,22.00
1598,99721024,^un^ Adjustable Hat,True,Embroidered/Sewn,100% Cotton,NaN,Sport Program,Imported,Basic Non-Seasonal,Legacy Apparel,^color^,99721139.0,Clothing & Accessories,Men,Hats,22.00


# Cleaning ^un^ in name field

In [10]:
def replace_data(data, word, str_replace):
    return data.replace(word, str_replace)

In [11]:
df_product['name'] =  df_product['name'].apply(lambda x: replace_data(x, '^un^', ''))

In [12]:
# Check for name field have '^un^'
df_product.query('name.str.contains("^un^")', engine='python')

,product_id,name,in_store,embellishment_type,fabric,fit,graphic_type,origin,season_code,vendor,color_code,single_sku_catalog_entry,catalog,category,subcategory,current_price


# Create Connect for import dataframe to RDS (Postgresql)

In [13]:
# Note: please change your database, username & password as per your own values
conn_params_dic = {
    "host"      : "localhost",
    "database"  : "local_test_noc",
    "user"      : "postgres",
    "password"  : "12345678"
}

# Using alchemy method
connect_alchemy = "postgresql+psycopg2://%s:%s@%s/%s" % (
    conn_params_dic['user'],
    conn_params_dic['password'],
    conn_params_dic['host'],
    conn_params_dic['database']
)

In [14]:
def using_alchemy(df, tablename):
    try:
        engine = create_engine(connect_alchemy)
        df.to_sql(tablename, con=engine, index=False, if_exists='append',chunksize = 1000)
        print("Data inserted using to_sql()(sqlalchemy) done successfully...")
    except OperationalError as err:
        # passing exception to function
        show_psycopg2_exception(err)

In [15]:
using_alchemy(df_custom, 'customer')

Data inserted using to_sql()(sqlalchemy) done successfully...


In [17]:
using_alchemy(df_product, 'product')

Data inserted using to_sql()(sqlalchemy) done successfully...


In [18]:
using_alchemy(df_netSale, 'net_sale')

Data inserted using to_sql()(sqlalchemy) done successfully...


# Call Function for calulate data

In [19]:
engine = create_engine(connect_alchemy)
result = engine.execute(text('call sum_net_total()').execution_options(autocommit=True))
result

# Result Table 

In [20]:
engine = create_engine(connect_alchemy)
df_total = pd.read_sql('select * from total_netsale',engine.connect())
df_total

,customer_id,first_name,last_name,total_sale_thb,shipping_thb,tax_thb,create_date,update_date
0,58598,Jaquenette,Gontier,13774.2,0.0,779.7,2022-11-14 22:57:34.919005,2022-11-14 22:57:34.919005
1,56760,Nancey,Collomosse,1132.5,179.4,56.1,2022-11-14 22:57:34.919005,2022-11-14 22:57:34.919005
2,55815,Far,Starzaker,729.9,150.0,35.4,2022-11-14 22:57:34.919005,2022-11-14 22:57:34.919005
3,11389,Raul,Alcorn,1443.9,229.2,68.7,2022-11-14 22:57:34.919005,2022-11-14 22:57:34.919005
4,14053,Robyn,Crane,859.8,150.0,27.3,2022-11-14 22:57:34.919005,2022-11-14 22:57:34.919005
...,...,...,...,...,...,...,...,...
7037,10556,Timotheus,Ferruzzi,3643.8,578.4,173.4,2022-11-14 22:57:34.919005,2022-11-14 22:57:34.919005
7038,58118,Alanson,Bragg,5221.2,0.0,307.2,2022-11-14 22:57:34.919005,2022-11-14 22:57:34.919005
7039,17891,Amaleta,Elmer,1443.9,229.2,68.7,2022-11-14 22:57:34.919005,2022-11-14 22:57:34.919005
7040,55912,Ty,Kyne,5380.8,845.7,306.6,2022-11-14 22:57:34.919005,2022-11-14 22:57:34.919005
